In [20]:
import os
import torch
import cv2 as cv
import numpy as np
import torchvision
from torchvision import transforms
import pandas as pd

In [21]:
# case can have to be mix of this two groops (rgb, dept, Ergbd, Lrgbd) and (rad, angle)

case = 'Prgbd_rad'
ver = 0

rgb_ver = 0

In [22]:
rgb_img_dir = './dataset/test/rgb/'
depth_img_dir = './dataset/test/depth/'

csv_path = './dataset/test/test_data.csv'

In [23]:
# Loads model.
def model_definition(rgb_weights, depth_weights):
    
    rgb_model = torchvision.models.resnet18(pretrained=False)
    rgb_model.fc = torch.nn.Linear(512, 1)
    
    depth_model = torchvision.models.resnet18(pretrained=False)
    depth_model.fc = torch.nn.Linear(512, 1)
    depth_model.conv1 = torch.nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)

    rgb_model.eval()
    depth_model.eval()
    
    rgb_model.load_state_dict(torch.load(rgb_weights)) # Load weights
    depth_model.load_state_dict(torch.load(depth_weights)) # Load weights

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    rgb_model.to(device)
    depth_model.to(device)

    return rgb_model, depth_model, device

# transfer image to tensor.
def im2tens(rgb, depth):
        
        rgb_transform = torchvision.transforms.ToTensor()
        depth_transform = torchvision.transforms.Compose([transforms.ToTensor(), transforms.Grayscale(num_output_channels=1)])
                 
        rgb_tensor = rgb_transform(rgb)
        depth_tensor = depth_transform(depth)

        return rgb_tensor, depth_tensor


# Pregict class.
def predict(img, model, device):
    
    xb = img.unsqueeze(0)
    yb = model(xb.to(device))
    dist = yb.cpu().detach().numpy()[0][0]
    
    return dist
            

def get_models(rgb_angle_weights, depth_angle_weights, rgb_rad_weights, depth_rad_weights):

    rgb_angle_model, depth_angle_model, device = model_definition(rgb_angle_weights, depth_angle_weights)
    rgb_rad_model, depth_rad_model, _ = model_definition(rgb_rad_weights, depth_rad_weights)

    return rgb_angle_model, depth_angle_model, rgb_rad_model, depth_rad_model, device

def main_pred(rgb_img, depth_img, rgb_angle_model, depth_angle_model, rgb_rad_model, depth_rad_model, device):
    
    rgb_tensor, depth_tensor = im2tens(rgb_img, depth_img)
    
    rgb_angle_pred = predict(rgb_tensor, rgb_angle_model, device)
    depth_angle_pred = predict(depth_tensor, depth_angle_model, device)
    rgb_rad_pred = predict(rgb_tensor, rgb_rad_model, device)
    depth_rad_pred = predict(depth_tensor, depth_rad_model, device)

    return rgb_angle_pred, depth_angle_pred, rgb_rad_pred, depth_rad_pred

In [24]:
rgb_angle_weights = './Weights/rgb_angle_v' + str(rgb_ver) + '.pth'
depth_angle_weights = './Weights/depth_angle_v' + str(rgb_ver) + '.pth'
rgb_rad_weights  = './Weights/rgb_rad_v' + str(rgb_ver) + '.pth'
depth_rad_weights  = './Weights/depth_rad_v' + str(rgb_ver) + '.pth'

rgb_angle_model, depth_angle_model, rgb_rad_model, depth_rad_model, device = get_models(rgb_angle_weights, depth_angle_weights, rgb_rad_weights, depth_rad_weights)

In [25]:
output_dir = './results/' + case.split('_')[0] + '_v' + str(ver) + '/'

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

output_csv_path = os.path.join(output_dir, 'graphs_data.csv')

df = pd.read_csv(csv_path)
im_name = df.loc[:, 'Picture name']
angle = df.loc[:, 'Angle [deg]']
rad = df.loc[:, 'Radial distance [mm]']

data = []
        
for ii in range(len(im_name)):
    
    rgb_img = cv.cvtColor(cv.imread(rgb_img_dir + im_name[ii]), cv.COLOR_BGR2RGB)
    depth_img = cv.cvtColor(cv.imread(depth_img_dir + im_name[ii]), cv.COLOR_BGR2RGB)
    rgb_angle_pred, depth_angle_pred, rgb_rad_pred, depth_rad_pred = main_pred(rgb_img, depth_img, rgb_angle_model, depth_angle_model, rgb_rad_model, depth_rad_model, device)    

    if np.abs(rgb_angle_pred - angle[ii]) > 180:
        rgb_angle_pred = 360 - rgb_angle_pred
        
    if np.abs(depth_angle_pred - angle[ii]) > 180:
        depth_angle_pred = 360 - depth_angle_pred

    angle_pred = (rgb_angle_pred+depth_angle_pred)/2
    rad_pred = (rgb_rad_pred+depth_rad_pred)/2

    if np.abs(angle_pred - angle[ii]) > 180:
        angle_pred = 360 - angle_pred
        
    data.append({
        'Picture name': im_name[ii],
        'gt_rad': rad[ii],
        'pred_rad': rad_pred,
        'gt_angle': angle[ii],
        'pred_angle': angle_pred
    })
    
# Convert the list of dictionaries to a DataFrame
data_df = pd.DataFrame(data)

# Save the DataFrame to a CSV file
data_df.to_csv(output_csv_path, index=False)